In [1]:
import subprocess
import shlex
import os
import json
import random
import io
import ast
from PIL import Image, ImageDraw, ImageFont

from openai import OpenAI
import base64



MAGIC_PDF_PATH = "/mnt/workspace/MinerU_Actual/mineru/bin/magic-pdf"


# --- 您的 magic-pdf 执行代码 ---
def run_magic_pdf(input_pdf_path, output_directory):
    """
    Args:
        input_pdf_path (str): 输入 PDF 文件的完整路径。
        output_directory (str): 输出文件的目录。

    Returns:
        tuple: (return_code, stdout, stderr) 命令行执行结果。
    """
    # 构建命令行指令，使用 magic-pdf 的绝对路径
    if not os.path.exists(MAGIC_PDF_PATH):
         print(f"错误：配置的 magic-pdf 路径不存在: {MAGIC_PDF_PATH}")
         print("请检查 MAGIC_PDF_PATH 变量是否指向了正确的 magic-pdf 可执行文件。")
         return 1, "", f"配置的 magic-pdf 路径不存在: {MAGIC_PDF_PATH}"

    # 检查输入文件是否存在
    if not os.path.exists(input_pdf_path):
        print(f"错误：输入 PDF 文件未找到于 {input_pdf_path}")
        return 1, "", f"输入 PDF 文件未找到于 {input_pdf_path}"

    # 确保输出目录存在
    try:
        os.makedirs(output_directory, exist_ok=True)
    except Exception as e:
        print(f"错误：创建输出目录失败 {output_directory}: {e}")
        return 1, "", f"创建输出目录失败 {output_directory}: {e}"


    command = f"{shlex.quote(MAGIC_PDF_PATH)} -p {shlex.quote(input_pdf_path)} -o {shlex.quote(output_directory)}"

    print(f"正在执行命令: {command}")

    try:
        # 使用 subprocess.run 执行命令
        result = subprocess.run(
            shlex.split(command), # 使用 shlex.split 安全分割命令字符串
            capture_output=True,
            text=True,
            check=True # 如果命令返回非零退出码，则抛出 CalledProcessError
        )

        print("命令执行成功。")
        print("stdout:", result.stdout)
        # print("stderr:", result.stderr) # 只有在 check=False 或出错时 stderr 才可能有内容

        return result.returncode, result.stdout, result.stderr

    except FileNotFoundError:
        #MAGIC_PDF_PATH 设置不正确
        error_msg = f"错误：找不到 magic-pdf 命令于 {MAGIC_PDF_PATH}。"
        print(error_msg)
        return 1, "", error_msg
    except subprocess.CalledProcessError as e:
        print(f"错误：命令执行失败，返回码 {e.returncode}")
        print("stdout:", e.stdout)
        print("stderr:", e.stderr)
        return e.returncode, e.stdout, e.stderr
    except Exception as e:
        print(f"执行命令时发生意外错误: {e}")
        return 1, "", str(e)



# @title Helper function for parsing JSON output (from a potentially markdown-fenced string)
def parse_json(json_output):
    """
    Parses JSON string, removing markdown fencing if present.
    """
    lines = json_output.splitlines()
    for i, line in enumerate(lines):
        if line.strip() == "```json":
            json_output = "\n".join(lines[i+1:])  # Remove everything before "```json"
            json_output = json_output.split("```")[0].strip()  # Remove everything after the closing "```"
            break  # Exit the loop once "```json" is found

        if line.strip().startswith('{') or line.strip().startswith('['):
             break
    return json_output



# @title Helper function for base64 encoding
def encode_image(image_path):
    """
    Encodes an image file to a base64 string.
    """
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

# @title inference function with API (This is the one used in process_json_with_images)
def inference_with_api(image_path, prompt, sys_prompt="You are a helpful assistant.", model_id="Qwen2.5-VL-7B-Instruct", min_pixels=512*28*28, max_pixels=2048*28*28):
    """
    Inference using an external API (OpenAI compatible).

    Args:
        image_path (str): Full path to the image file.
        prompt (str): The prompt for the API.
        sys_prompt (str): The system prompt.
        model_id (str): The model identifier.
        min_pixels (int): Minimum pixel constraint.
        max_pixels (int): Maximum pixel constraint.

    Returns:
        str: The text response from the API.

    Raises:
        Exception: If API call fails.
    """
    print(f"--- Calling inference_with_api for {os.path.basename(image_path)} ---")
    try:
        base64_image = encode_image(image_path)
    except Exception as e:
        print(f"错误：编码图片 {image_path} 失败: {e}")
        raise

    llm_cfg = {
        'model': model_id, # 使用传入的 model_id
        'model_server': 'http://localhost:8000/v1',  # 本地服务器地址
        'api_key': 'EMPTY',  # 本地服务器无需 API Key
        'generate_cfg': {
            'top_p': 0.8,
            'max_tokens': 1024 # 注意这里的 max_tokens 可能会限制返回的文本长度
        }
    }

    try:
        client = OpenAI(
            api_key=llm_cfg['api_key'],
            base_url=llm_cfg['model_server']
            # 如果使用 Dashscope 官方服务，这里需要配置实际 API Key 和 Base URL
            # api_key=os.getenv('DASHSCOPE_API_KEY'),
            # base_url="[https://dashscope-intl.aliyuncs.com/compatible-mode/v1](https://dashscope-intl.aliyuncs.com/compatible-mode/v1)",
        )

        # 确定图片格式，这里假设都是 jpeg，如果实际情况多样，需要更鲁棒的判断
        image_format = 'jpeg' # 或者从文件扩展名判断 img_full_path.split('.')[-1]

        messages=[
            {
                "role": "system",
                "content": [{"type":"text","text": sys_prompt}]},
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        # 根据需要调整 min_pixels, max_pixels
                        # "min_pixels": min_pixels,
                        # "max_pixels": max_pixels,
                        "image_url": {"url": f"data:image/{image_format};base64,{base64_image}"},
                    },
                    {"type": "text", "text": prompt},
                ],
            }
        ]
        # print("API Request Messages:", messages) # 调试用

        completion = client.chat.completions.create(
            model = llm_cfg['model'], # 使用 llm_cfg 中的 model 名称
            messages = messages,
            # 可以将 llm_cfg['generate_cfg'] 中的参数传递给 create 方法
            # top_p=llm_cfg['generate_cfg'].get('top_p'),
            # max_tokens=llm_cfg['generate_cfg'].get('max_tokens'),
        )
        # print("API Response:", completion) # 调试用

        if completion and completion.choices and completion.choices[0].message:
             return completion.choices[0].message.content
        else:
             print("警告：API 调用返回空或非预期结构。")
             return "API 返回空结果"

    except Exception as e:
        print(f"错误：调用 API 进行推理失败: {e}")

        raise # 重新抛出异常以便 process_json_with_images 捕获处理

# --- 您的 JSON 处理主函数 ---
def process_json_with_images(json_filepath):
    """
    读取JSON文件，处理图片块，添加img_text字段，并保存修改后的JSON。

    Args:
        json_filepath (str): 输入的JSON文件路径。
    """
    if not os.path.exists(json_filepath):
        print(f"错误：JSON 文件未找到于 {json_filepath}")
        return

    # 获取JSON文件所在的目录，用于构建图片完整路径

    json_dir = os.path.dirname(os.path.abspath(json_filepath))
    print(f"正在处理 JSON 文件: {json_filepath}")
    print(f"JSON 文件目录: {json_dir}")

    try:
        # 加载JSON数据
        with open(json_filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except json.JSONDecodeError as e:
        print(f"错误：解码 JSON 失败: {e}")
        return
    except Exception as e:
        print(f"错误：读取 JSON 文件失败: {e}")
        return

    # 假设data是一个列表，遍历其中的每一个块
    if not isinstance(data, list):
        print("警告：JSON 数据不是列表。脚本预期顶层 JSON 是一个块列表。退出。")
        return

    blocks_list = data
    modified_count = 0

    # 遍历所有块
    for i, block in enumerate(blocks_list):
        # 检查块类型是否为image
        if isinstance(block, dict) and block.get("type") == "image":
            img_relative_path = block.get("img_path")

            if img_relative_path:
                # 构建完整的图片文件路径
                # img_relative_path 通常是 "images/..."
                # os.path.join(json_dir, img_relative_path) 会正确构建到图片文件的完整路径
                img_full_path = os.path.join(json_dir, img_relative_path)

                print(f"\n发现图片块（索引 {i}）。正在处理图片: {img_full_path}")

                # 检查图片文件是否存在
                if not os.path.exists(img_full_path):
                    print(f"警告：图片文件未找到于 {img_full_path}。跳过此块处理。")
                    block["img_text"] = f"错误：图片文件未找到于 {img_full_path}"
                    modified_count += 1
                    continue # 跳过当前块的进一步处理

                try:


                    # 定义 prompt
                    prompt = "请识别出图中所有的文字"


                    response = inference_with_api(img_full_path, prompt)

                    print(f"图片（索引 {i}）的文本结果:\n---\n{response}\n---")

                    # 将文本结果添加到当前JSON块的img_text字段中
                    block["img_text"] = response
                    modified_count += 1
                    print(f"已为图片块（索引 {i}）添加 'img_text'。")

                except Exception as e:
                    print(f"错误：处理图片 {img_full_path}（索引 {i}）时出错: {e}")
                    # 在出错时也添加一个错误信息到 img_text
                    block["img_text"] = f"错误：处理图片时出错: {e}"
                    modified_count += 1


    # 构建输出文件路径 (例如在原文件名后加上 _processed)
    base, ext = os.path.splitext(json_filepath)
    output_filepath = f"{base}_processed{ext}"

    # 保存修改后的数据到新的JSON文件
    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            # 使用 indent 使输出的JSON文件更易读
            # ensure_ascii=False 来正确处理非ASCII字符（如中文）
            json.dump(data, f, indent=4, ensure_ascii=False)
        print(f"\n处理完成。共处理了 {modified_count} 个图片块。")
        print(f"修改后的 JSON 已保存至: {output_filepath}")
    except Exception as e:
        print(f"错误：保存修改后的 JSON 文件失败: {e}")



In [3]:

if __name__ == "__main__":

    input_pdf_file = "/mnt/workspace/Qwen-Agent/test_files/output_page4.pdf"

    output_base_dir = "/mnt/workspace/MinerU_Output"

    print(f"输入 PDF 文件: {input_pdf_file}")
    print(f"magic-pdf 输出根目录 (-o 参数): {output_base_dir}")

    # 运行 magic-pdf 进行解析
    print("\n--- 阶段 1: 运行 magic-pdf 解析 PDF ---")
    return_code, stdout, stderr = run_magic_pdf(input_pdf_file, output_base_dir)

    if return_code == 0:
        print("\n--- magic-pdf 解析成功 ---")
        pdf_filename_without_ext = os.path.splitext(os.path.basename(input_pdf_file))[0]
        generated_json_file = os.path.join(
            output_base_dir,
            pdf_filename_without_ext,
            "auto",
            f"{pdf_filename_without_ext}_content_list.json"
        )

        print(f"\n--- 阶段 2: 处理生成的 JSON 文件 ---")
        print(f"将要处理的 JSON 文件（根据 magic-pdf 输出结构构建）: {generated_json_file}")

        if os.path.exists(generated_json_file):
             # 调用 JSON 处理函数
             process_json_with_images(generated_json_file)
        else:
             print(f"错误：预期的 JSON 文件未找到于 {generated_json_file}")
             print("magic-pdf 可能执行失败或生成了非预期的文件结构。")


    else:
        print("\n--- magic-pdf 解析失败，跳过 JSON 处理 ---")
        print("错误信息:")
        print(stderr)


输入 PDF 文件: /mnt/workspace/Qwen-Agent/test_files/output_page4.pdf
magic-pdf 输出根目录 (-o 参数): /mnt/workspace/MinerU_Output

--- 阶段 1: 运行 magic-pdf 解析 PDF ---
正在执行命令: /mnt/workspace/MinerU_Actual/mineru/bin/magic-pdf -p /mnt/workspace/Qwen-Agent/test_files/output_page4.pdf -o /mnt/workspace/MinerU_Output
命令执行成功。
stdout: 

--- magic-pdf 解析成功 ---

--- 阶段 2: 处理生成的 JSON 文件 ---
将要处理的 JSON 文件（根据 magic-pdf 输出结构构建）: /mnt/workspace/MinerU_Output/output_page4/auto/output_page4_content_list.json
正在处理 JSON 文件: /mnt/workspace/MinerU_Output/output_page4/auto/output_page4_content_list.json
JSON 文件目录: /mnt/workspace/MinerU_Output/output_page4/auto

发现图片块（索引 9）。正在处理图片: /mnt/workspace/MinerU_Output/output_page4/auto/images/e0982325b1972679ab4649579d202a6d2d9443ae6fea314596d7b7cd941cbb6c.jpg
--- Calling inference_with_api for e0982325b1972679ab4649579d202a6d2d9443ae6fea314596d7b7cd941cbb6c.jpg ---
图片（索引 9）的文本结果:
---
logit
Prediction Head
User LLM
E1
E2
...
En
Etgt
(a). Early Fusion
logit
Prediction Head
User 